# A2A リモートエージェント対応のマルチエージェントシステム

## 事前準備

In [ ]:
%pip install --upgrade --user google-adk

In [19]:
# カーネル再起動
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

In [1]:
import copy, json, os, uuid
import vertexai
from vertexai import agent_engines

from google.genai.types import Part, Content
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.agents.sequential_agent import SequentialAgent

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
[PROJECT_NUMBER] = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

/home/jupyter/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(


In [2]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if DEBUG:
                print(f'====\n{event}\n====')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## リモートエージェントの定義

In [4]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
指定されたテーマの記事を執筆する際に参考となるトピックを５項目程度のリストにまとめます。
後段のエージェントがこのリストに基づいて、調査レポートを作成します。

* 出力形式
日本語で出力。
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    instruction=instruction,
)

In [5]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
前段のエージェントは、５項目程度の調査対象トピックを指定します。

* 出力形式
日本語で出力。
調査レポートは、トピックごとに客観的情報をまとめます。各トピックについて、５文以上の長さで記述すること。
'''

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    instruction=instruction,
)

In [6]:
instruction = '''
あなたの役割は、特定のテーマに関する気軽な読み物記事を書くことです。
記事の「テーマ」と、その内容に関連する「調査レポート」が与えられるので、
調査レポートに記載の客観的事実に基づいて、信頼性のある読み物記事を書いてください。

レビュアーによる修正ポイントが指摘された際は、直前に書いた記事を指摘に従って書き直してください。

**出力条件**
- トピックに関してある程度の基礎知識がある読者を前提として、数分で気軽に読める内容にしてください。
- 比較的カジュアルで語りかける口調の文章にします。
- 思考過程は出力せずに、最終結果だけを出力します。
- 記事タイトルは付けないで、次の構成で出力します。各セクションタイトルは、内容に合わせてアレンジしてください。
0. 導入：セクションタイトルを付けないで、この記事を読みたくなる導入を１〜２文にまとめます。
1. 概要：トピックの全体像をまとめて簡単に説明します。
2. 最新情報：特に注目したい新しい情報を取り上げます。
3. 実践：トピックに関して、読者自身がやってみるとよさそうな事を１つ紹介します。
4. まとめ

- 各セクションのタイトルはマークダウンヘッダ ## 付けます。必要に応じて箇条書きのマークダウンを使用します。
- それ以外のマークダウンによる装飾は行いません。
'''

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='特定のテーマに関する読み物記事を書くエージェント',
    instruction=instruction,
)

In [7]:
instruction = f'''
あなたの役割は、読み物記事をレビューして、記事の条件にあった内容にするための改善コメントを与える事です。

* 記事の条件
- 記事は、はじめに４０文字程度のタイトルがあること。
　今日から役立つ生活情報があって「すぐに読まなきゃ」と読者が感じるタイトルにすること。
　タイトルはマークダウンヘッダ # をつけること。
- タイトルの直後に「なぜいまこのテーマを取り上げるのか」をまとめた導入を加えて、読者にこの記事を読む動機づけを与えます。
- 各セクションのサブタイトルには、絵文字を用いて親しみやすさを出すこと。
- 読者が今日から実践できる具体例が３つ以上紹介されていること。

* 出力形式
- 日本語で出力。
- はじめに、記事の良い点を説明します。
- 次に、修正ポイントを箇条書きで出力します。
'''

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='読み物記事をレビューするエージェント',
    instruction=instruction,
)

## Agent Engine へのデプロイ

In [ ]:
agents = {
    'research_agent1': research_agent1,
    'research_agent2': research_agent2,
    'writer_agent': writer_agent,
    'review_agent': review_agent,
}

deployed_agents = [agent.display_name for agent in agent_engines.list()]

for agent_name in agents.keys():
    if not target in deployed_agents:
        remote_agent = agent_engines.create(
            agent_engine=agents[agent_name],
            display_name=f'{agent_name}',
            requirements=['google-adk']
        )
    else:
        resource_name = [
            agent.name for agent in agent_engines.list()
            if agent.display_name == agent_name
        ][0]
        remote_agent = agent_engines.update(
            resource_name=resource_name,
            agent_engine=agents[agent_name],
            display_name=f'{agent_name}',
            requirements=['google-adk']
        )

In [20]:
agent_list = list(agent_engines.list())
for agent_name in agents.keys():
    resource_name = [
        agent.name for agent in agent_list
        if agent.display_name == agent_name
    ][0]
    print(f'{agent_name}: {resource_name}')

research_agent1: 1229051689714057216
research_agent2: 4103474151883276288
writer_agent: 3456081705448767488
review_agent: 1011753007693430784


## A2A サーバーのデプロイ

In [29]:
%%bash
cat <<EOF >main.py
import os
import vertexai
from vertexai import agent_engines

from a2a.server.agent_execution import AgentExecutor
from a2a.server.tasks import TaskUpdater, InMemoryTaskStore
from a2a.types import (
    Task, TextPart, UnsupportedOperationError,
    AgentCapabilities, AgentCard, AgentSkill,
)
from a2a.utils.errors import ServerError
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler

PROJECT_ID = os.environ['PROJECT_ID']
LOCATION = os.environ['REGION']
AGENT_ID = os.environ['AGENT_ID']
SERVICE_URL = os.environ['SERVICE_URL']

vertexai.init(project=PROJECT_ID, location=LOCATION)


class AdkAgent:
    SUPPORTED_CONTENT_TYPES = ['text', 'text/plain']

    def __init__(self, remote_agent):
        self._remote_agent = remote_agent

    async def stream(self, updater, query):
        session = self._remote_agent.create_session(user_id='default_user')
        events = self._remote_agent.stream_query(
            user_id='default_user',
            session_id=session['id'],
            message=query,
        )
        text_parts = []        
        for event in events:
            if text_parts: # not a last part
                updater.new_agent_message(text_parts)
            text_parts = []
            if 'content' in event and 'parts' in event['content']:
                for part in event['content']['parts']:
                    if 'text' in part:
                        text_parts.append(TextPart(text=part['text']))
        # treat last part as an artifact
        await updater.add_artifact(text_parts)
        remote_agent.delete_session(user_id='default_user', session_id=session['id'])


class AdkAgentExecutor(AgentExecutor):
    def __init__(self, adk_agent: AdkAgent):
        self.adk_agent = adk_agent

    async def cancel(self, request, event_queue):
        raise ServerError(error=UnsupportedOperationError('Cancel operation is not supported.'))

    async def execute(self, context, event_queue):
        updater = TaskUpdater(event_queue, context.task_id, context.context_id)
        if not context.current_task:
            await updater.submit()
        await updater.start_work()

        query = context.get_user_input()
        await self.adk_agent.stream(updater, query)
        await updater.complete()


def create_agent_card(
    base_url, supported_content_types, agent_name, agent_description):
    
    capabilities = AgentCapabilities(streaming=True)
    skill = AgentSkill(
        id=f"{agent_name.lower().replace(' ', '_')}_skill",
        name=f'{agent_name} Skill',
        description=agent_description,
        tags=[tag.strip() for tag in agent_name.lower().split()],
        examples=[f'Interact with {agent_name}'],
    )

    return AgentCard(
        name=agent_name,
        description=agent_description,
        url=base_url,
        version='1.0.0',
        defaultInputModes=supported_content_types,
        defaultOutputModes=supported_content_types,
        capabilities=capabilities,
        skills=[skill],
    )


remote_agent = agent_engines.get(AGENT_ID)
adk_agent = AdkAgent(remote_agent=remote_agent)

agent_card = create_agent_card(
    base_url=SERVICE_URL,
    supported_content_types=AdkAgent.SUPPORTED_CONTENT_TYPES,
    agent_name=remote_agent.name,
    agent_description=remote_agent.name,
)
    
request_handler = DefaultRequestHandler(
    agent_executor=AdkAgentExecutor(adk_agent),
    task_store=InMemoryTaskStore(),
)

server_app_builder = A2AStarletteApplication(
    agent_card=agent_card, http_handler=request_handler
)

app = server_app_builder.build()
EOF

cat <<EOF >Dockerfile
FROM python:3.11-slim
WORKDIR /app
ENV PYTHONDONTWRITEBYTECODE 1
ENV PYTHONUNBUFFERED 1
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]
EOF

cat <<EOF >requirements.txt
uvicorn
starlette
google-adk==1.10.0
google-cloud-aiplatform==1.108.0
a2a-sdk==0.3.1
EOF

In [ ]:
REPO_NAME='cloud-run-source-deploy'
REPO=f'{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}'
!gcloud artifacts repositories create {REPO_NAME} \
     --repository-format docker --location {LOCATION}

In [ ]:
agents = [
    'research_agent1',
    'research_agent2',
    'writer_agent',
    'review_agent',
]

for agent_name in agents:
    AGENT_ID = deployed_agents[agent_name]
    print(f'## {agent_name}: {AGENT_ID}')

    SERVICE_NAME=f'a2a-server-{agent_name}'.replace('_', '-')
    SERVICE_URL=f'https://{SERVICE_NAME}-{PROJECT_NUMBER}.{LOCATION}.run.app'

    !mkdir -p {agent_name}
    !cp main.py requirements.txt Dockerfile {agent_name}/
    !cd {agent_name} && gcloud builds submit --tag {REPO}/{SERVICE_NAME}
    !gcloud run deploy {SERVICE_NAME} \
       --image {REPO}/{SERVICE_NAME} \
       --platform managed \
       --region {LOCATION} \
       --no-allow-unauthenticated \
       --update-env-vars "\
PROJECT_ID={PROJECT_ID},REGION={LOCATION},AGENT_ID={AGENT_ID},\
SERVICE_URL={SERVICE_URL}" 2>&1 | cat

## リモートエージェント対応版・業務フローエージェントの定義

In [61]:
import httpx
from a2a.client import A2AClient
from a2a.types import MessageSendParams, SendStreamingMessageRequest
import google.auth.transport.requests
import google.oauth2.id_token 


def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
         return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )

    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )


async def a2a_remote_call(query, a2a_server_url):
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, a2a_server_url)
    headers = {
        'Authorization': f'Bearer {id_token}',
        'Content-Type': 'application/json'
    }
    payload = {
        'message': {
            'role': 'user',
            'parts': [{'kind': 'text', 'text': query}],
            'contextId': uuid.uuid4().hex,
            'messageId': uuid.uuid4().hex,
        },
    }
    request = SendStreamingMessageRequest(params=MessageSendParams(**payload))

    async with httpx.AsyncClient(headers=headers, timeout=100) as httpx_client:
        client = await A2AClient.get_client_from_agent_card_url(
            httpx_client, a2a_server_url
        )
        stream_response = client.send_message_streaming(request)
        result = []
        async for chunk in stream_response:
            chunk = json.loads(chunk.root.model_dump_json(exclude_none=True))
            if chunk['result']['kind'] == 'artifact-update':
                text_messages = []
                for part in chunk['result']['artifact']['parts']:
                    if 'text' in part:
                        text_messages.append(part['text'])
                result.append(Part(text='\n'.join(text_messages)))
    
    return result


def get_call_a2a_agent(agent_name):
    service_name=f'a2a-server-{agent_name}'.replace('_', '-')
    a2a_server_url=f'https://{service_name}-{PROJECT_NUMBER}.{LOCATION}.run.app'

    async def call_a2a_agent(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> Content:
        parts = await a2a_remote_call(str(llm_request.contents), a2a_server_url)
        return LlmResponse(
            content=Content(role='model', parts=parts) 
        )
    return call_a2a_agent


## Proxy Agents
research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash', # not used
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    before_model_callback=get_call_a2a_agent('research_agent1'),
)

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    before_model_callback=get_call_a2a_agent('research_agent2'),
)

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='特定のテーマに関する読み物記事を書くエージェント',
    before_model_callback=get_call_a2a_agent('writer_agent'),
)

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='読み物記事をレビューするエージェント',
    before_model_callback=get_call_a2a_agent('review_agent'),
)
####

research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## リサーチエージェントが調査レポートを作成します。\n---\n'),
        get_print_agent('\n## 調査対象のトピックを選定します。\n'),
        copy.deepcopy(research_agent1),
        get_print_agent('\n## 選定したトピックに基づいて、調査レポートを作成します。\n'),
        copy.deepcopy(research_agent2),
        get_print_agent('\n#### 調査レポートが準備できました。記事の作成に取り掛かってもよいでしょうか？\n'),
    ],
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント',
)

write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## ライターエージェントが記事を執筆します。\n---\n'),
        copy.deepcopy(writer_agent),
        get_print_agent('\n---\n## レビューエージェントが記事をレビューします。\n---\n'),
        copy.deepcopy(review_agent),
       get_print_agent('\n#### レビューに基づいて記事の修正を依頼しますか？\n'),
    ],
    description='記事を作成、レビューする。',
)

root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
何ができるか聞かれた場合は、以下の処理をすることをわかりやすくフレンドリーな文章にまとめて返答してください。

- ユーザーが指定したテーマの記事を作成する業務フローを実行する。
- はじめに、テーマに関する調査レポートを作成する。
- その後、ライターエージェントとレビューエージェントが協力して、編集方針に則した記事を作成する。

ユーザーが記事のテーマを指定した場合は、次のフローを実行します。

1. そのテーマの記事の作成に取り掛かる旨を伝えて、research_agent に転送して、調査レポートを依頼します。
2. ユーザー記事の作成を支持したら、write_and_review_agent に転送して、記事の作成とレビューを依頼します。
3. ユーザーが記事の修正を希望する場合は、write_and_review_agent に転送します。

**条件**
research_agent のニックネームは、リサーチエージェント
write_and_review_agent のネックネームは、ライターエージェントとレビューエージェント

''',
    sub_agents=[
        copy.deepcopy(research_agent),
        copy.deepcopy(write_and_review_agent),
    ],
    description='記事を作成する業務フローを実行するエージェント'
)

## 実行例

In [62]:
client = LocalApp(root_agent)

DEBUG=False
query = '''
こんにちは。何ができますか？
'''
result = await client.stream(query)

こんにちは！私は記事作成の業務フローを実行するエージェントです。具体的には、ユーザー様が指定されたテーマについて、

1.  まず、リサーチエージェントがテーマに関する調査レポートを作成します。
2.  その後、ライターエージェントとレビューエージェントが協力して、編集方針に沿った記事を作成します。

という流れで記事を作成できます。何かご希望のテーマはありますか？



In [63]:
query = '''
「夏の電気代節約術」をテーマに書いてください。
'''
result = await client.stream(query)

承知いたしました。「夏の電気代節約術」ですね。記事の作成に取り掛かります！まずは、リサーチエージェントに調査レポートを依頼します。



---
## リサーチエージェントが調査レポートを作成します。
---


## 調査対象のトピックを選定します。

承知いたしました。「夏の電気代節約術」に関する記事執筆のための調査レポート作成に向け、以下の5つのトピックを選定しました。後段のエージェントがこのリストに基づき、詳細な調査を進めます。

---
## 調査対象のトピックリスト

1.  **エアコンの効率的な利用法**: 設定温度、風量、フィルター清掃、自動運転機能の活用など、エアコンの電気代を抑える具体的な方法。
2.  **家電製品の見直しと節電術**: 冷蔵庫、照明、テレビなど、エアコン以外の主要家電における待機電力削減や効率的な使用法、買い替えの検討。
3.  **住まいの断熱・遮熱対策**: 窓や壁からの熱侵入を防ぐための遮光カーテン、すだれ、グリーンカーテン、断熱材の活用といった物理的な対策。
4.  **電力契約・料金プランの見直し**: 契約アンペア数の適正化、時間帯別料金プランの活用、新電力への切り替えなど、契約面からの節電アプローチ。
5.  **夏の生活習慣と身体の冷却**: 扇風機やミストの活用、打ち水、涼しい服装、入浴方法など、電気に頼らず涼しく過ごすためのライフスタイルの工夫。

---

## 選定したトピックに基づいて、調査レポートを作成します。

## 夏の電気代節約術に関する調査レポート

### 1. エアコンの効率的な利用法

エアコンは夏の電気代を大きく左右する家電であり、その効率的な利用は節電において非常に重要です。まず、設定温度は環境省が推奨する28℃を目安に設定することが望ましいとされています。室温を1℃上げるだけで、約10%の消費電力削減につながるとも言われています。また、風量は「自動運転」に設定することで、部屋の温度変化に応じて最も効率の良い運転モードをエアコンが選択するため、消費電力を抑えられます。さらに、エアコンのフィルターは2週間に1回程度の清掃が推奨されており、これにより通気性が向上し、年間で約25%もの消費電力量削減が見込めるとされています。扇風機やサーキュレーターをエアコンと併用することで、冷たい空気

In [64]:
query = '''
はい。お願いします。
'''
result = await client.stream(query)

承知いたしました。それでは、ライターエージェントとレビューエージェントに記事の作成とレビューを依頼します。



---
## ライターエージェントが記事を執筆します。
---

うだるような日本の夏、エアコンは手放せないけど、毎月の電気代の請求書を見るたびに冷や汗をかいていませんか？ 今年こそ、賢く快適に夏を乗り切るための電気代節約術を一緒に見ていきましょう！

## 賢い夏の電気代節約術、全体像

夏の電気代を抑えるには、実はエアコンだけが対象じゃないってご存知でしたか？ 大きく分けて、以下の5つの視点からアプローチするのがおすすめです。

*   **エアコンの効率的な利用**: 設定温度の見直しやフィルター清掃など、日々の使い方で大きく変わります。
*   **家電製品の見直しと節電**: 冷蔵庫や照明、テレビといったエアコン以外の家電も、使い方次第で節電の余地がたっぷり。
*   **住まいの断熱・遮熱対策**: 窓や壁からの熱の侵入を防ぐことで、エアコンに頼りすぎない涼しさを実現できます。
*   **電力契約・料金プランの見直し**: 電気の「使い方」だけでなく、「契約」そのものを最適化することも非常に効果的です。
*   **夏の生活習慣と身体の冷却**: 電気を使わずに涼しく過ごすための昔ながらの知恵や、ちょっとした工夫も侮れません。

これらの対策を組み合わせることで、今年の夏はきっと快適に、そしてお財布に優しく過ごせるはずですよ。

## 電力プラン見直しが新しい常識？！

電気代節約と聞くと、まずは「エアコンの設定温度を下げる」とか「こまめに電気を消す」といった行動を思い浮かべがちですよね。もちろんそれらも大切ですが、最近特に注目されているのが「電力契約や料金プランの見直し」なんです。

2016年の電力自由化以降、様々な新しい電力会社が登場し、多様な料金プランが提供されています。例えば、ライフスタイルに合わせて夜間の電気代が安くなる「時間帯別料金プラン」を利用したり、ご自身の電力使用量に合ったアンペア数に契約し直すだけで、基本料金が節約できることも。
「ウチは昔からの電力会社だから」と固定観念を持たずに、一度各社のウェブサイトで料金シミュレーションを試してみるのがおすすめです。もしかしたら、今の契約よりもずっとお得なプランが見つかるかもしれ

In [65]:
query = '''
はい。お願いします。
'''
result = await client.stream(query)

承知いたしました。それでは、レビューエージェントからの指摘に基づき、ライターエージェントに記事の修正を依頼します。



---
## ライターエージェントが記事を執筆します。
---

# 夏の電気代を賢くカット！快適に過ごす節約術2024

うだるような日本の夏、エアコンは手放せないけど、毎月の電気代の請求書を見るたびに冷や汗をかいていませんか？ 今年こそ、賢く快適に夏を乗り切るための電気代節約術を一緒に見ていきましょう！

## 賢い夏の電気代節約術、全体像💡

夏の電気代を抑えるには、実はエアコンだけが対象じゃないってご存知でしたか？ 大きく分けて、以下の5つの視点からアプローチするのがおすすめです。

*   **エアコンの効率的な利用**: 設定温度の見直しやフィルター清掃など、日々の使い方で大きく変わります。扇風機やサーキュレーターとの併用で体感温度を下げられるだけでなく、冷気を効率よく循環させて設定温度を無理なく上げられますよ。
*   **家電製品の見直しと節電**: 冷蔵庫や照明、テレビといったエアコン以外の家電も、使い方次第で節電の余地がたっぷり。特に冷蔵庫は、中身を詰め込みすぎない、開閉時間を短くする、設定温度を「中」にするだけでも大きな節約になります。また、使わない家電のコンセントを抜く「待機電力カット」もお忘れなく！
*   **住まいの断熱・遮熱対策**: 窓や壁からの熱の侵入を防ぐことで、エアコンに頼りすぎない涼しさを実現できます。遮光カーテンやすだれ、グリーンカーテンなども効果的ですよ。
*   **電力契約・料金プランの見直し**: 電気の「使い方」だけでなく、「契約」そのものを最適化することも非常に効果的です。
*   **夏の生活習慣と身体の冷却**: 電気を使わずに涼しく過ごすための昔ながらの知恵や、ちょっとした工夫も侮れません。打ち水や涼しい服装、ひんやり寝具なども活用しましょう。

これらの対策を組み合わせることで、今年の夏はきっと快適に、そしてお財布に優しく過ごせるはずですよ。

## 電力プラン見直しが新しい常識？！✨

電気代節約と聞くと、まずは「エアコンの設定温度を下げる」とか「こまめに電気を消す」といった行動を思い浮かべがちですよね。もちろんそれらも大切ですが、最近特に注目されているのが「電力契約や料金プランの

In [66]:
query = f'''
これで完成です。完成した記事をもう一度表示して。
'''
result = await client.stream(query)

はい、承知いたしました。完成した記事はこちらです。

# 夏の電気代を賢くカット！快適に過ごす節約術2024

うだるような日本の夏、エアコンは手放せないけど、毎月の電気代の請求書を見るたびに冷や汗をかいていませんか？ 今年こそ、賢く快適に夏を乗り切るための電気代節約術を一緒に見ていきましょう！

## 賢い夏の電気代節約術、全体像💡

夏の電気代を抑えるには、実はエアコンだけが対象じゃないってご存知でしたか？ 大きく分けて、以下の5つの視点からアプローチするのがおすすめです。

*   **エアコンの効率的な利用**: 設定温度の見直しやフィルター清掃など、日々の使い方で大きく変わります。扇風機やサーキュレーターとの併用で体感温度を下げられるだけでなく、冷気を効率よく循環させて設定温度を無理なく上げられますよ。
*   **家電製品の見直しと節電**: 冷蔵庫や照明、テレビといったエアコン以外の家電も、使い方次第で節電の余地がたっぷり。特に冷蔵庫は、中身を詰め込みすぎない、開閉時間を短くする、設定温度を「中」にするだけでも大きな節約になります。また、使わない家電のコンセントを抜く「待機電力カット」もお忘れなく！
*   **住まいの断熱・遮熱対策**: 窓や壁からの熱の侵入を防ぐことで、エアコンに頼りすぎない涼しさを実現できます。遮光カーテンやすだれ、グリーンカーテンなども効果的ですよ。
*   **電力契約・料金プランの見直し**: 電気の「使い方」だけでなく、「契約」そのものを最適化することも非常に効果的です。
*   **夏の生活習慣と身体の冷却**: 電気を使わずに涼しく過ごすための昔ながらの知恵や、ちょっとした工夫も侮れません。打ち水や涼しい服装、ひんやり寝具なども活用しましょう。

これらの対策を組み合わせることで、今年の夏はきっと快適に、そしてお財布に優しく過ごせるはずですよ。

## 電力プラン見直しが新しい常識？！✨

電気代節約と聞くと、まずは「エアコンの設定温度を下げる」とか「こまめに電気を消す」といった行動を思い浮かべがちですよね。もちろんそれらも大切ですが、最近特に注目されているのが「電力契約や料金プランの見直し」なんです。

2016年の電力自由化以降、様々な新しい電力会社が登場し、多様な料金プランが提供されています。例えば、ライフスタ